In [6]:
import pandas as pd
PATH_REPORTS = '../reports/'
lstm_results_path = 'test_results/LSTM_with_Attention_{asset}_test_results.csv'
mlp_results_path =  'test_results/MLP_{asset}_test_results.csv'

In [7]:
ASSETS = ["PETR3.SA", "PRIO3.SA", "VALE3.SA", "GGBR3.SA", "ABCB4.SA", "ITUB3.SA", "FLRY3.SA", "RADL3.SA"]

In [12]:
for asset in ASSETS:
    
    lstm_results = pd.read_csv(PATH_REPORTS + lstm_results_path.format(asset = asset.replace(".", "_")), index_col = 0)
    mlp_results = pd.read_csv(PATH_REPORTS + mlp_results_path.format(asset = asset.replace(".", "_")), index_col = 0)
    
    print(asset)

PETR3.SA
PRIO3.SA
VALE3.SA
GGBR3.SA
ABCB4.SA
ITUB3.SA
FLRY3.SA
RADL3.SA


In [15]:
lstm_results.iloc[0,0][0]

'['

In [16]:
lstm_results

,y_test,y_pred
0,[-0.1283354],-0.026802
1,[0.18002863],0.025595
2,[0.93982095],0.254662
3,[1.11934331],0.784867
4,[1.48142243],0.903238
...,...,...
959,[1.30153032],1.011319
960,[1.89789484],1.034221
961,[1.74071159],1.411713
962,[1.70210743],1.303902
